# Assignment: Extracting Static WebPage

Extract information about “วันพระ” for 3 years from:
- https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2565.aspx
- https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2566.aspx
- https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2567.aspx


Note that you can use dateparse package to parse Thai date.  First, we will have to install the package, this is for Google Colab users.  Otherwise, installing via command line is recommended (pip or conda).

In [15]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    %pip install dateparser

In [16]:
import dateparser

To convert from thai text date string, we will use the parse method.  Note that the parse method assumes the BC year, not BE.  Thus, we will have to subtract 543 from the year.  In addition, weekday() returns day of week with 0=Monday, ..., 6=Sunday.

In [17]:
dt = dateparser.parse('วันศุกร์ที่ 17 มกราคม 2563')

# this will print out weekday == 0 (Monday)
print(dt)
print(dt.weekday())

# this will print out weekday == 4 (Friday)
dt = dt.replace(year=dt.year-543)
print(dt)
print(dt.weekday())

2563-01-17 00:00:00
0
2020-01-17 00:00:00
4


In [18]:
dt = dateparser.parse('วันเสาร์ที่ 21 กันยายน 2564')
dt = dt.replace(year=dt.year-543)
print(dt)
print(dt.weekday())

2021-09-21 00:00:00
1


Count the distribution of number of week days that are “วันพระ” for all three years and answer the following questions:

In [19]:
import requests

data_2565 = requests.get('https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2565.aspx')
data_2566 = requests.get('https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2566.aspx')
data_2567 = requests.get('https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2567.aspx')

# https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2565.aspx
# https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2566.aspx
# https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2567.aspx

In [20]:
day_dict = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}

In [21]:
import pandas as pd

day = [
    i.strip(
        "</div><div class='bud-day-col'></div></div><div class='cls'></div><!--www.myhora.com-->"
    ).strip("")
    for i in "".join(
        [
            i
            for i in [
                i.strip()
                for i in (
                    str(data_2565.text) + str(data_2566.text) + str(data_2567.text)
                ).split("\n")
            ]
            if "<div class='bud-day-col'>" in i
        ]
    ).split("<div class='bud-day'><div class='bud-day-col'>")
]
clean_day = []
for d in day:
    l = d.find("<")
    if l >= 0:
        clean_day.append(d[:l])

clean_data = [
    {
        "text": d,
        "date": dateparser.parse(d).replace(year=dateparser.parse(d).year - 543),
        "week_day": day_dict[dateparser.parse(d).weekday()],
    }
    for d in clean_day
]

df = pd.DataFrame(clean_data, columns=["text", "date", "week_day"])

df


,text,date,week_day
0,วันอาทิตย์ที่ 2 มกราคม 2565,2022-01-02,Wednesday
1,วันจันทร์ที่ 10 มกราคม 2565,2022-01-10,Thursday
2,วันจันทร์ที่ 17 มกราคม 2565,2022-01-17,Thursday
3,วันอังคารที่ 25 มกราคม 2565,2022-01-25,Friday
4,วันอังคารที่ 1 กุมภาพันธ์ 2565,2022-02-01,Friday
...,...,...,...
147,วันเสาร์ที่ 30 พฤศจิกายน 2567,2024-11-30,Monday
148,วันอาทิตย์ที่ 8 ธันวาคม 2567,2024-12-08,Tuesday
149,วันอาทิตย์ที่ 15 ธันวาคม 2567,2024-12-15,Tuesday
150,วันจันทร์ที่ 23 ธันวาคม 2567,2024-12-23,Wednesday


## How many วันพระ in total (of 3 years)?

In [22]:
df.shape[0]

152

## How many days in total (of 3 years) that วันพระ is Monday?

In [23]:
df[df.week_day == "Monday"].shape[0]

21

## Which day of the week that has the minimum number of วันพระ?

In [24]:
max_wanpra_week_day = df['week_day'].value_counts()[lambda x: x == df['week_day'].value_counts().min()].index.tolist()
for d in max_wanpra_week_day:
    print(d)

Friday


## Which day of the week that has the maximum number of วันพระ?

In [25]:
min_wanpra_week_day = df['week_day'].value_counts()[lambda x: x == df['week_day'].value_counts().max()].index.tolist()
for d in min_wanpra_week_day:
    print(d)

Sunday
Tuesday
